In [1]:
!pip install transformers==4.15.0 sentencepiece
!pip install datasets==1.17.0 
!pip install pythainlp
!pip install jiwer
!pip install sklearn-crfsuite
!pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 15.6 MB/s 
     |████████████████████████████████| 1.3 MB 41.3 MB/s 
     |████████████████████████████████| 880 kB 41.7 MB/s 
     |████████████████████████████████| 120 kB 57.1 MB/s 
     |████████████████████████████████| 3.3 MB 65.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1eb98a6466be6ae302b3f2305a11f8fd7218aacdda749cd330364de65d3837a8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 306 kB 17.3 MB/s 
     |████████████████████████████████| 115 kB 66.5 MB/s 
     |████████████████████████████████| 212 kB 74.3 MB/s 
Looking in indexes: https://pypi.org/simp

In [2]:
from transformers import AutoModelForMaskedLM, pipeline
from transformers import AutoTokenizer, BertForTokenClassification, AutoModel
import pandas as pd
import torch
import pickle
from tqdm import tqdm
from datasets import load_metric
from pythainlp.benchmarks import word_tokenization
from nltk.translate.gleu_score import sentence_gleu
from jiwer import cer

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [3]:
#Data analysis
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
# from sklearn_crfsuite import CRF, scorers, metrics
# import sklearn_crfsuite
# from sklearn_crfsuite import scorers
# from sklearn_crfsuite import metrics
# from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import scipy.stats
# import eli5

In [4]:
from itertools import chain

import nltk
import sklearn
import scipy.stats

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite import metrics

In [5]:
# tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', model_max_length=512)
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/json/tokenizer_json_15k.pkl', 'rb'))

In [ ]:
tokenizer_json = pickle.load(open('drive/MyDrive/AIBuilders/json/tokenizer_json_15k.pkl', 'rb'))

In [6]:
class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        return output

In [ ]:
tagging_model = BertModel()
# model = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
# model.resize_token_embeddings(len(tokenizer_th))

# FILE = "drive/MyDrive/AIBuilders/tagging.pth"
FILE = "drive/MyDrive/AIBuilders/json/tagging_json_400.pth"
tagging_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
tagging_model.eval()

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.8.attention.self.query.weight', 'roberta.encoder.layer.3.attention.self.key.weight', 'roberta.encoder.layer.9.output.LayerNorm.weight', 'roberta.encoder.layer.11.output.dense.bias', 'roberta.encoder.layer.8.attention.self.value.weight', 'roberta.encoder.layer.6.output.LayerNorm.bias', 'roberta.encoder.layer.8.attention.output.dense.bias', 'roberta.embeddings.position_embeddings.weight', 'roberta.encoder.layer.5.intermediate.dense.weight', 'roberta.encoder.layer.11.attention.self.value.weight', 'lm_head.layer_norm.bias', 'roberta.encoder.layer.1.attention.self.query.weight', 'roberta.encoder.layer.11.attention.self.key.weight', 'roberta.encoder.layer.10.output.dense.bias', 'roberta.encoder.layer.5.intermediate.dense.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.dense

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(25354, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [ ]:
# tagging_model.bert.resize_token_embeddings(len(tokenizer))

In [ ]:
ids_to_labels = {0: 'f', 1: 'i'}

def evaluate_one_text(model, sentence, mask, labels):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    input_id = torch.Tensor([sentence]).type(torch.int64)
    label_ids = []
    for i in sentence:
      if i == 1 or i == 5 or i == 6:
        label_ids.append(-100)
      else:
        label_ids.append(2)
    label_ids = torch.Tensor([label_ids]).type(torch.int64)
    mask = torch.Tensor([mask]).type(torch.int64)

    logits = tagging_model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    return prediction_label

In [ ]:
mlm_model = AutoModelForMaskedLM.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
mlm_model.resize_token_embeddings(len(tokenizer))

FILE = "drive/MyDrive/AIBuilders/json/mlm_json_6.pth"
mlm_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
mlm_model = mlm_model.to(device)
mlm_model.eval()

CamembertForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25354, 768)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

In [ ]:
# mlm_model.resize_token_embeddings(len(tokenizer))

In [ ]:
ds_tag = pickle.load(open('drive/MyDrive/AIBuilders/json/ner_json_15k_2.pkl', 'rb'))
ds_mlm = pickle.load(open('drive/MyDrive/AIBuilders/json/mlm_json_15k_2.pkl', 'rb'))

In [ ]:
df_mlm_test = ds_mlm[10576:]
df_mlm = ds_mlm[:10576]
df_mlm_train, df_mlm_val = np.split(df_mlm.sample(frac=1, random_state=42),
                            [int(.9 * len(df_mlm))])

df_tag_test = ds_tag[10576:]
df_tag = ds_tag[:10576]
df_tag_train, df_tag_val = np.split(df_tag.sample(frac=1, random_state=42),
                            [int(.9 * len(df_tag))])

# ds_mlm = ds_mlm[10576:]
# ds_tag = ds_tag[10576:]
# NUM_SAMPLE = ds_mlm.shape[0]
# NUM_SAMPLE

In [ ]:
df_tag_train

,text,labels
1284,"{'input_ids': [[tensor(5), tensor(5248), tenso...","[tensor(0), tensor(0), tensor(0), tensor(1), t..."
6997,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
9188,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
5076,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
1393,"{'input_ids': [[tensor(5), tensor(330), tensor...","[tensor(0), tensor(0), tensor(0), tensor(1), t..."
...,...,...
4757,"{'input_ids': [[tensor(5), tensor(627), tensor...","[tensor(0), tensor(0), tensor(0), tensor(1), t..."
10098,"{'input_ids': [[tensor(5), tensor(729), tensor...","[tensor(0), tensor(0), tensor(1), tensor(0), t..."
6320,"{'input_ids': [[tensor(5), tensor(7894), tenso...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
5975,"{'input_ids': [[tensor(5), tensor(2169), tenso...","[tensor(0), tensor(0), tensor(1), tensor(0), t..."


In [ ]:
df_tag_train.iloc[0]['labels']

tensor([   0,    0,    0,    1,    1,    0, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -1

In [ ]:
def make_crf_data(df):
  output = []
  for i in tqdm(range(len(df))):
    sentence = []
    row = df.iloc[i]['text']['input_ids'][0]
    label = df.iloc[i]['labels']
    for j in range(len(row)):
      token = row[j]
      label_token = label[j]
      if token not in [1,5,6]:
        word = tokenizer.convert_ids_to_tokens([token])
        item = (word[0], "AA", str(int(label_token)))
        sentence.append(item)
        # print(item)
    output.append(sentence)
  return output

In [ ]:
crf_tag_train = make_crf_data(df_tag_train)

100%|██████████| 9518/9518 [00:47<00:00, 199.08it/s]


In [ ]:
crf_tag_test = make_crf_data(df_tag_test)

100%|██████████| 5000/5000 [00:29<00:00, 167.42it/s]


In [ ]:
crf_tag_train

[[('▁ทํา', 'AA', '0'),
  ('ไม่', 'AA', '0'),
  ('เปน', 'AA', '1'),
  ('อะ', 'AA', '1')],
 [('▁', 'AA', '0'),
  ('ติดต่อ', 'AA', '0'),
  ('ธนาคาร', 'AA', '0'),
  ('เรียบร้อย', 'AA', '0'),
  ('แร้ว', 'AA', '1'),
  ('ค', 'AA', '1'),
  ('๊ะ', 'AA', '1')],
 [('▁', 'AA', '0'),
  ('โทร', 'AA', '0'),
  ('กลับไป', 'AA', '0'),
  ('ไม่ได้', 'AA', '0'),
  ('ค่ะ', 'AA', '0'),
  ('▁มี', 'AA', '0'),
  ('วิธีอื่น', 'AA', '0'),
  ('อีกมั้ยคะ', 'AA', '0'),
  ('▁', 'AA', '0'),
  ('ให้น้อง', 'AA', '0'),
  ('โทรไป', 'AA', '0'),
  ('ทําให้เขา', 'AA', '0'),
  ('ก้', 'AA', '1'),
  ('บอก', 'AA', '0'),
  ('ไม่ใช่', 'AA', '0'),
  ('เจ้าของ', 'AA', '0'),
  ('บัตร', 'AA', '0'),
  ('▁นี่', 'AA', '0'),
  ('ใช้จ่าย', 'AA', '0'),
  ('อะไร', 'AA', '0'),
  ('ไม่ได้', 'AA', '0'),
  ('เรย', 'AA', '1')],
 [('▁', 'AA', '0'),
  ('จะทํา', 'AA', '0'),
  ('รายการที่', 'AA', '0'),
  ('โอน', 'AA', '0'),
  ('เง', 'AA', '1'),
  ('ืน', 'AA', '1'),
  ('ผ่าน', 'AA', '0'),
  ('มือถือ', 'AA', '0'),
  ('ได้', 'AA', '0'),
  ('อ่ะ', 'AA', 

In [ ]:
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

In [ ]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

In [ ]:
#Creating the train and test set
X = [sent2features(s) for s in crf_tag_train]
y = [sent2labels(s) for s in crf_tag_train]

In [ ]:
X_test = [sent2features(s) for s in crf_tag_test]
y_test = [sent2labels(s) for s in crf_tag_test]

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
)
crf.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [ ]:
labels = list(crf.classes_)
# labels.remove('O')
labels

['0', '1']

In [ ]:
len(y_pred)

5000

In [ ]:
cou = 0
all = 0
for i in tqdm(range(len(y_pred))):
# for i in tqdm(range(1000)):
  a = y_pred[i]
  # print("aaaa")
  # print(len(a))
  # print(a)
  label = df_tag_test.iloc[i]['labels']
  label = [str(int(k)) for k in label if k != -100]
  # print(label[1:-1])
  # print(len(label))
  for j in range(1, len(label)-1):
      label_token = label[j]
      # print(j)
      if label_token != -100:
        all += 1
        if a[j-1] == label_token:
          cou += 1
print(all, cou, cou/all)

100%|██████████| 5000/5000 [00:15<00:00, 328.32it/s]

44937 43094 0.9589870262812382


In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.9586703289455609

In [ ]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['0', '1'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

           0      0.969     0.980     0.974     35235
           1      0.922     0.884     0.903      9702

    accuracy                          0.959     44937
   macro avg      0.945     0.932     0.939     44937
weighted avg      0.959     0.959     0.959     44937



In [ ]:
#We predcit using the same 5 fold cross validation
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
ds_tag.iloc[1153].values

array([{'input_ids': tensor([[    5,    10,   803,  6069,   367, 25320,    10,  1355,  3748,  1360,
                   20,  2144,     6,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
                    1,     1,     1,    

In [ ]:
msp_type = pickle.load(open('drive/MyDrive/AIBuilders/json/msp_type.pkl', 'rb'))
msp_type = msp_type[10576:]
msp_word = pickle.load(open('drive/MyDrive/AIBuilders/json/msp_word.pkl', 'rb'))
msp_word = msp_word[10576:]
print(len(msp_type), len(msp_word))

5000 5000


In [ ]:
msp_type

[[['morphed', 10, 11]],
 [['misspelled', 1, 2]],
 [['misspelled', 1, 2]],
 [['misspelled', 4, 6]],
 [['misspelled', 1, 2], ['morphed', 3, 5]],
 [['morphed', 1, 2], ['misspelled', 3, 4]],
 [['misspelled', 6, 7]],
 [['misspelled', 2, 4]],
 [['misspelled', 4, 5]],
 [['misspelled', 3, 4]],
 [['misspelled', 1, 4]],
 [['misspelled', 3, 4], ['morphed', 5, 6]],
 [['misspelled', 1, 2], ['misspelled', 4, 5]],
 [['misspelled', 3, 4]],
 [['misspelled', 2, 3], ['morphed', 7, 8]],
 [['morphed', 3, 4]],
 [['misspelled', 3, 6]],
 [['morphed', 4, 7]],
 [['misspelled', 1, 4]],
 [['morphed', 6, 7], ['morphed', 13, 14], ['misspelled', 15, 16]],
 [['misspelled', 2, 3]],
 [['morphed', 3, 4]],
 [['misspelled', 1, 3], ['misspelled', 4, 5]],
 [['morphed', 3, 4]],
 [['misspelled', 6, 7]],
 [['misspelled', 7, 8]],
 [['misspelled', 2, 4]],
 [['misspelled', 4, 6]],
 [['misspelled', 2, 3]],
 [['misspelled', 4, 5]],
 [['misspelled', 3, 4]],
 [['misspelled', 1, 2], ['morphed', 12, 13]],
 [['misspelled', 4, 9]],
 [['m

In [ ]:
msp_type[965:975]

[[['morphed', 1, 2]],
 [['morphed', 15, 16]],
 [['misspelled', 1, 2]],
 [['morphed', 9, 10]],
 [['misspelled', 1, 7]],
 [['morphed', 5, 6]],
 [['misspelled', 5, 6]],
 [['morphed', 4, 5]],
 [['misspelled', 19, 21]],
 [['misspelled', 1, 2]]]

In [ ]:
for i in range(100):
  print(msp_type[i])
  print(msp_word[i])
  print("------------------")

[['morphed', 10, 11]]
[[('มั๊ย', 'ไหม')]]
------------------
[['misspelled', 1, 2]]
[[('ก้', 'ก็')]]
------------------
[['misspelled', 1, 2]]
[[('อยุ่', 'อยู่')]]
------------------
[['misspelled', 4, 6]]
[[('ตุ้', 'ตู้')]]
------------------
[['misspelled', 1, 2], ['morphed', 3, 5]]
[[('แอฟ', 'แอพ')], [('หรอ', 'หรือ'), ('ค่ะ', 'คะ')]]
------------------
[['morphed', 1, 2], ['misspelled', 3, 4]]
[[('คับ', 'ครับ')], [('รุ้', 'รู้')]]
------------------
[['misspelled', 6, 7]]
[[('คับ', 'ครับ')]]
------------------
[['misspelled', 2, 4]]
[[('เบอร', 'เบอร์')]]
------------------
[['misspelled', 4, 5]]
[[('ใหน', 'ไหน')]]
------------------
[['misspelled', 3, 4]]
[[('อัพเดต', 'อัปเดต')]]
------------------
[['misspelled', 1, 4]]
[[('เช้ค', 'เช็ค')]]
------------------
[['misspelled', 3, 4], ['morphed', 5, 6]]
[[('เบอ', 'เบอร์')], [('คับ', 'ครับ')]]
------------------
[['misspelled', 1, 2], ['misspelled', 4, 5]]
[[('แอฟ', 'แอป')], [('อยาก', 'ยาก')]]
------------------
[['misspelled', 3, 4]]


In [ ]:
def ids_to_tokens(tokenized_text):
  a = tokenizer.convert_ids_to_tokens(tokenized_text)
  # a.remove("<s>")
  # a.remove("</s>")
  # if a[0] == '▁':
  #   a.pop(0)
  return a

In [ ]:
msp_type_dict = {}
msp_type_dict_full = {}
msp_type_dict_fill = {}
sent_id_type = {}
msp_word_dict = {}
msp_word_dict_full = {}
msp_word_dict_fill = {}

NUM_SAMPLE = 5000
len_msp_type = len(msp_type)
# for i in range(len_msp_type):
for i in range(NUM_SAMPLE):
  len_a = len(msp_type[i])
  for j in range(len_a):
    k = msp_type[i][j][0]
    if not k in msp_type_dict:
      msp_type_dict[k] = 1
      msp_type_dict_full[k] = 0
      msp_type_dict_fill[k] = 0
      sent_id_type[k] = []
    else:
      msp_type_dict[k] += 1
    
    len_b = len(msp_word[i][j])
    for k in range(len_b):
      x = msp_word[i][j][k]
      if not x in msp_word_dict:
        msp_word_dict[x] = 1
        msp_word_dict_full[x] = 0
        msp_word_dict_fill[x] = 0
      else:
        msp_word_dict[x] += 1

msp_word_dict_wrong = msp_word_dict_fill.copy()
print(msp_type_dict)
print(msp_type_dict_fill)
print(msp_type_dict_full)
msp_word_dict = dict(sorted(msp_word_dict.items(), key=lambda item: item[1], reverse=True))
print(msp_word_dict)
print(msp_word_dict_fill)
print(msp_word_dict_full)

bleu = load_metric("bleu")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

for sent_id in tqdm(range(NUM_SAMPLE)):
  chng = []
  text = ds_tag.iloc[sent_id]['text']['input_ids'].squeeze(0).tolist()
  mask = ds_mlm.iloc[sent_id]['attention_mask']
  labels = ds_mlm.iloc[sent_id]['labels']
  text = [k for k in text if k != 1]
  labels = [k for k in labels if k != 1]
  mask = [k for k in mask if k != 0]
  # print(text)
  # print(labels)
  # print(mask)
  original = [ids_to_tokens(text)]
  references = [[ids_to_tokens(labels)]]
  moi = ''.join(original[0])
  moi = moi.replace("▁", " ")
  # print(f"TEXT: {moi}")
  

  i_f = evaluate_one_text(tagging_model, text, mask, labels)
  predicted_id = text.copy()
  original_id = text.copy()
  i_f_len = len(i_f)
  for j in range(i_f_len):
    if(i_f[j] == 'i'):
      ph = predicted_id[j+1]

      predicted_id[j+1] = 25004
      mlm_input = {'input_ids': torch.Tensor([predicted_id]).type(torch.int64).to(device), 'attention_mask': torch.Tensor([mask]).type(torch.int64).to(device)}
      token_logits = mlm_model(**mlm_input).logits
      mask_token_index = torch.where(mlm_input["input_ids"] == tokenizer.mask_token_id)[1]
      mask_token_logits = token_logits[0, mask_token_index, :]
      top_5_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()
      # print(f"{tokenizer.convert_ids_to_tokens(ph)} ({ph}) => {tokenizer.convert_ids_to_tokens(top_5_tokens[0])} ({top_5_tokens[0]})")
      chng.append((j, top_5_tokens[0]))

      predicted_id[j+1] = ph

  for x,y in chng:
    predicted_id[x+1] = y

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  labels_len = len(labels)
  for i in range(labels_len):
    if not predicted_id[i] == original_id[i]: #change
      denom += 1
      if predicted_id[i] == labels[i]:
        numer += 1
        TP += 1
      elif not predicted_id[i] == labels[i]:
        FP += 1
    elif predicted_id[i] == original_id[i]: #no change
      if predicted_id[i] == labels[i]:
        TN += 1
      elif not predicted_id[i] == labels[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1
  

  ans = tokenizer.convert_ids_to_tokens(predicted_id)
  # ans.remove('<s>')
  # ans.remove('</s>')
  if ans[0] == '▁':
    ans.pop(0)
  predictions = [ans]  
  ans = ''.join(ans)
  ans = ans.replace("▁", " ")

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)
  
  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  # print(references)
  # print(predictions)
  # print(original)
  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred

  # print(f"MSP TYPE: {msp_type[sent_id]}")
  for i in msp_type[sent_id]:
    msp_type_dict_full[i[0]] += 1
    flag = 0
    a = []
    for j in range(i[1]+1, i[2]+1):
      a.append((original_id[j], predicted_id[j]))
      # print(tokenizer.convert_ids_to_tokens(predicted_id[j]), tokenizer.convert_ids_to_tokens(labels[j]))
      if predicted_id[j] != labels[j]:
        flag = 1
        break
    if flag == 0:
      msp_type_dict_fill[i[0]] += 1
      if sent_id not in sent_id_type[i[0]]: #it may look too little because it only takes full marks
        sent_id_type[i[0]].append([sent_id, a])


  for i in range(labels_len):
    if labels[i] != original_id[i]: #change
      word_original = tokenizer.convert_ids_to_tokens(original_id[i])
      word_original = word_original.replace("▁", "")
      word_predicted = tokenizer.convert_ids_to_tokens(predicted_id[i])
      word_predicted = word_predicted.replace("▁", "")
      word_labels = tokenizer.convert_ids_to_tokens(labels[i])
      word_labels = word_labels.replace("▁", "")
      # if (word_original, word_predicted) in msp_word_dict_fill: #have key
      if (word_original, word_labels) in msp_word_dict_fill: #have key
        # msp_word_dict_full[(word_original, word_predicted)] += 1
        msp_word_dict_full[(word_original, word_labels)] += 1
        if labels[i] == predicted_id[i]: #correct
          # msp_word_dict_fill[(word_original, word_predicted)] += 1
          msp_word_dict_fill[(word_original, word_labels)] += 1
        elif labels[i] != predicted_id[i]: #incorrect
          # msp_word_dict_wrong[(word_original, word_predicted)] += 1
          msp_word_dict_wrong[(word_original, word_labels)] += 1

 


  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")
  # print("--------------------------------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")
print(msp_type_dict_full)
print(msp_type_dict_fill)
msp_word_dict_full = dict(sorted(msp_word_dict_full.items(), key=lambda item: item[1], reverse=True))
msp_word_dict_fill = dict(sorted(msp_word_dict_fill.items(), key=lambda item: item[1], reverse=True))
msp_word_dict_wrong = dict(sorted(msp_word_dict_wrong.items(), key=lambda item: item[1], reverse=True))
print(msp_word_dict_full)
print(msp_word_dict_fill)
print(msp_word_dict_wrong)
print(sent_id_type)

{'morphed': 1818, 'misspelled': 4509, 'ws': 36, 'other': 38, 'abbreviation': 190, 'new': 1}
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{'morphed': 0, 'misspelled': 0, 'ws': 0, 'other': 0, 'abbreviation': 0, 'new': 0}
{('อ่ะ', 'อะ'): 626, ('ค่ะ', 'คะ'): 487, ('คับ', 'ครับ'): 466, ('คะ', 'ค่ะ'): 345, ('เบอ', 'เบอร์'): 315, ('มั้ย', 'ไหม'): 205, ('สมัค', 'สมัคร'): 186, ('แอพ', 'แอป'): 170, ('เปน', 'เป็น'): 153, ('ค้ะ', 'คะ'): 148, ('ค้ะ', 'ค่ะ'): 146, ('อ่อ', 'อ๋อ'): 143, ('ยังไง', 'อย่างไร'): 136, ('ก้', 'ก็'): 135, ('ใด้', 'ได้'): 106, ('คัฟ', 'ครับ'): 90, ('มั๊ย', 'ไหม'): 88, ('เรย', 'เลย'): 70, ('โทรศัพ', 'โทรศัพท์'): 70, ('ใหม', 'ไหม'): 68, ('ไง', 'อย่างไร'): 67, ('ไม', 'ไหม'): 67, ('เคดิต', 'เครดิต'): 62, ('บช', 'บัญชี'): 56, ('แอฟ', 'แอป'): 55, ('ตัง', 'สตางค์'): 55, ('ม่', 'ไม่'): 55, ('อยุ่', 'อยู่'): 53, ('บันชี', 'บัญชี'): 53, ('ไหม่', 'ใหม่'): 51, ('อ้ะ', 'อะ'): 49, ('รุ้', 'รู้'): 48, ('ก้อ', 'ก็'): 48, ('แร้ว', 'แล้ว'): 46, ('ใหน', 'ไหน

Downloading:   0%|          | 0.00/2.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

100%|██████████| 5000/5000 [25:13<00:00,  3.30it/s]

AVG ACC: 0.5345659523809524
AVG f1: 0.5315847992530338
# HIGHER BLEU PREDICTION: 2852
# LOWER BLEU PREDICTION: 383
# EQUAL BLEU PREDICTION: 1765
# GLEU PREDICTION: 0.7476152588010968
# GLEU ORIGINAL: 0.5936627613265107
# CER PREDICTION: 0.08526382445948132
# CER ORIGINAL: 0.08266901295858524
{'morphed': 1818, 'misspelled': 4509, 'ws': 36, 'other': 38, 'abbreviation': 190, 'new': 1}
{'morphed': 868, 'misspelled': 2147, 'ws': 6, 'other': 14, 'abbreviation': 54, 'new': 0}
{('อ่ะ', 'อะ'): 626, ('ค่ะ', 'คะ'): 489, ('คับ', 'ครับ'): 467, ('เบอ', 'เบอร์'): 352, ('คะ', 'ค่ะ'): 345, ('มั้ย', 'ไหม'): 205, ('แอพ', 'แอป'): 170, ('เปน', 'เป็น'): 153, ('โทร', 'โทรศัพท์'): 150, ('ค้ะ', 'คะ'): 148, ('ค้ะ', 'ค่ะ'): 146, ('อ่อ', 'อ๋อ'): 143, ('ยังไง', 'อย่างไร'): 137, ('ก้', 'ก็'): 135, ('ใด้', 'ได้'): 106, ('คัฟ', 'ครับ'): 90, ('มั๊ย', 'ไหม'): 88, ('เรย', 'เลย'): 70, ('ใหม', 'ไหม'): 68, ('ไง', 'อย่างไร'): 67, ('ไม', 'ไหม'): 67, ('บช', 'บัญชี'): 56, ('แอฟ', 'แอป'): 55, ('ตัง', 'สตางค์'): 55, ('ม่', 'ไม่'

In [ ]:
pickle_model_json = {}
pickle_model_json['msp_type_dict_full'] = msp_type_dict_full
pickle_model_json['msp_type_dict_fill'] = msp_type_dict_fill
pickle_model_json['msp_word_dict_full'] = msp_word_dict_full
pickle_model_json['msp_word_dict_fill'] = msp_word_dict_fill
pickle_model_json['msp_word_dict_wrong'] = msp_word_dict_wrong
pickle_model_json['sent_id_type'] = sent_id_type

pickle.dump(pickle_model_json, open('pickle_model_json.pkl', 'wb'))

In [ ]:
pickle_model_json['msp_type_dict_full']

{'abbreviation': 190,
 'misspelled': 4509,
 'morphed': 1818,
 'new': 1,
 'other': 38,
 'ws': 36}

In [ ]:
for key in sent_id_type:
  print(f"=========[{key}]==========")
  for sent_id, words in sent_id_type[key]:
    text = ds_tag.iloc[sent_id]['text']['input_ids'].squeeze(0).tolist()
    labels = ds_mlm.iloc[sent_id]['labels']
    text = [k for k in text if k != 1]
    labels = [k for k in labels if k != 1]
    original = [ids_to_tokens(text)]
    references = [[ids_to_tokens(labels)]]
    for i in words:
      print(f"MSP: {tokenizer.convert_ids_to_tokens(i[0])} => {tokenizer.convert_ids_to_tokens(i[1])}")
    print(original)
    print(references)
    print("-------------------")

Streaming output truncated to the last 5000 lines.
MSP: เปน => เป็น
[['<s>', '▁ถ้า', 'เปน', 'แบบนี้', 'ทําไงดี', 'คะ', '</s>']]
[[['<s>', '▁ถ้า', 'เป็น', 'แบบนี้', 'ทําไงดี', 'คะ', '</s>']]]
-------------------
MSP: เบอ => เบอร์
[['<s>', '▁', 'เปลี่ยน', 'เบอ', 'ตรงไหน', 'หรอ', 'คะ', '▁', 'หาไม่เจอ', 'ค่ะ', '</s>']]
[[['<s>', '▁', 'เปลี่ยน', 'เบอร์', 'ตรงไหน', 'เหรอ', 'คะ', '▁', 'หาไม่เจอ', 'ค่ะ', '</s>']]]
-------------------
MSP: ค่ะ => คะ
[['<s>', '▁พี่', 'ค่ะ', '.', '</s>']]
[[['<s>', '▁พี่', 'คะ', '.', '</s>']]]
-------------------
MSP: แอพ => แอป
[['<s>', '▁', 'เปลี่ยน', 'มือถือ', 'ละ', 'เข้า', 'แอพ', 'scb', 'ไม่ได้', '▁', 'เครียด', 'เด้อ', '</s>']]
[[['<s>', '▁', 'เปลี่ยน', 'มือถือ', 'ละ', 'เข้า', 'แอป', 'scb', 'ไม่ได้', '▁', 'เครียด', 'เด้อ', '</s>']]]
-------------------
MSP: คะ => ค่ะ
[['<s>', '▁', 'อยากสอบถาม', 'เรื่อง', 'โอน', 'ตัง', 'เข้า', 'พร้อมเพย์', '▁', 'คะ', '</s>']]
[[['<s>', '▁', 'อยากสอบถาม', 'เรื่อง', 'โอน', 'สตางค์', 'เข้า', 'พร้อมเพย์', '▁', 'ค่ะ', '</s>']]]
---

In [ ]:
pickle_model_json = pickle.load(open('drive/MyDrive/AIBuilders/json/pickle_model_json.pkl', 'rb'))
msp_type_dict_full = pickle_model_json['msp_type_dict_full']
msp_type_dict_fill = pickle_model_json['msp_type_dict_fill']
msp_word_dict_full = pickle_model_json['msp_word_dict_full']
msp_word_dict_fill = pickle_model_json['msp_word_dict_fill']
msp_word_dict_wrong = pickle_model_json['msp_word_dict_wrong']
sent_id_type = pickle_model_json['sent_id_type']

In [ ]:
pickle_model_json

{'msp_type_dict_fill': {'abbreviation': 54,
  'misspelled': 2147,
  'morphed': 868,
  'new': 0,
  'other': 14,
  'ws': 6},
 'msp_type_dict_full': {'abbreviation': 190,
  'misspelled': 4509,
  'morphed': 1818,
  'new': 1,
  'other': 38,
  'ws': 36},
 'msp_word_dict_fill': {(' ', '.'): 0,
  ('.', '.'): 0,
  ('K My play', 'K My pay'): 0,
  ('k', 'k plus'): 0,
  ('k plus', 'k plus'): 0,
  ('plus', 'k plus'): 0,
  ('plusอ่ะคับ', 'อะครับ'): 0,
  ('ก', 'ก'): 0,
  ('ก กรุงไทย', 'กรุงไทย'): 0,
  ('กรุงเทพฯ', 'กรุงเทพ'): 0,
  ('กสิกรช', 'กสิกร'): 0,
  ('กอีก', 'อีก'): 0,
  ('กะ', 'กับ'): 2,
  ('กะ', 'ก็'): 1,
  ('กะตัง', 'สตางค์'): 0,
  ('กับ', 'กลับ'): 0,
  ('กับ', 'กัน'): 0,
  ('กัล', 'กัน'): 0,
  ('กาก', 'การ'): 0,
  ('กุ', 'กู'): 0,
  ('กุ้', 'กู้'): 0,
  ('ก็เรย', 'ก็เลย'): 0,
  ('ก่า', 'กว่า'): 0,
  ('ก้', 'ก็'): 113,
  ('ก้คือ', 'ก็คือ'): 0,
  ('ก้ดี', 'ก็ดี'): 0,
  ('ก้วิธีการ', 'ก็วิธีการ'): 0,
  ('ก้อ', 'ก็'): 32,
  ('ก้เลย', 'ก็เลย'): 0,
  ('ขอ', 'ของ'): 0,
  ('ขอบคุน', 'ขอบคุณ'): 30,

In [ ]:
percent_correct = {}
for key in msp_word_dict_full:
  fill = msp_word_dict_fill[key]
  full = msp_word_dict_full[key]
  if full == 0 or full < 30:
    percent_correct[key] = 0
  else:
    percent_correct[key] = fill/full
a = dict(sorted(percent_correct.items(), key=lambda item: item[1], reverse=True))
print(a)

{('มั้ย', 'ไหม'): 0.9560975609756097, ('ไง', 'อย่างไร'): 0.9253731343283582, ('อ่อ', 'อ๋อ'): 0.916083916083916, ('มั๊ย', 'ไหม'): 0.9090909090909091, ('ม่', 'ไม่'): 0.9090909090909091, ('ค้ะ', 'คะ'): 0.8851351351351351, ('ไม', 'ไหม'): 0.8805970149253731, ('เบอ', 'เบอร์'): 0.8721590909090909, ('ใหม', 'ไหม'): 0.8676470588235294, ('ใด้', 'ได้'): 0.8584905660377359, ('เปน', 'เป็น'): 0.8431372549019608, ('ก้', 'ก็'): 0.837037037037037, ('แอฟ', 'แอป'): 0.8363636363636363, ('รุ้', 'รู้'): 0.8333333333333334, ('อ้ะ', 'อะ'): 0.7959183673469388, ('อยุ่', 'อยู่'): 0.7924528301886793, ('ค่ะ', 'คะ'): 0.7893660531697342, ('ยังไง', 'อย่างไร'): 0.7883211678832117, ('อ่ะ', 'อะ'): 0.7875399361022364, ('ใม่', 'ไม่'): 0.78125, ('บช', 'บัญชี'): 0.7678571428571429, ('ใหน', 'ไหน'): 0.7674418604651163, ('ขอบคุน', 'ขอบคุณ'): 0.75, ('ค้ะ', 'ค่ะ'): 0.726027397260274, ('คะ', 'ค่ะ'): 0.7130434782608696, ('เค้า', 'เขา'): 0.6842105263157895, ('ตัง', 'สตางค์'): 0.6727272727272727, ('ก้อ', 'ก็'): 0.6666666666666666, ('

In [ ]:
percent_wrong = {}
for key in msp_word_dict_full:
  fill = msp_word_dict_wrong[key]
  full = msp_word_dict_full[key]
  if full == 0 or full < 30:
    percent_wrong[key] = 0
  else:
    percent_wrong[key] = fill/full
a = dict(sorted(percent_wrong.items(), key=lambda item: item[1], reverse=True))
print(a)

{('อะ', 'อ่ะ'): 1.0, ('ป่ะ', 'เปล่า'): 1.0, ('ป่าว', 'เปล่า'): 1.0, ('แอฟ', 'แอพ'): 1.0, ('โทร', 'โทรศัพท์'): 0.9933333333333333, ('หรอ', 'เหรอ'): 0.8918918918918919, ('หรอ', 'หรือ'): 0.8780487804878049, ('ค', 'ค่ะ'): 0.8545454545454545, ('ใช่', 'ใช้'): 0.8125, ('คัฟ', 'ครับ'): 0.7222222222222222, ('ค่า', 'ค่ะ'): 0.6451612903225806, ('คับ', 'ครับ'): 0.6209850107066381, ('เรย', 'เลย'): 0.6142857142857143, ('แร้ว', 'แล้ว'): 0.5652173913043478, ('ค', 'คะ'): 0.4, ('ไช่', 'ใช่'): 0.3548387096774194, ('แอพ', 'แอป'): 0.35294117647058826, ('ก้อ', 'ก็'): 0.3333333333333333, ('ตัง', 'สตางค์'): 0.32727272727272727, ('เค้า', 'เขา'): 0.3157894736842105, ('คะ', 'ค่ะ'): 0.28695652173913044, ('ค้ะ', 'ค่ะ'): 0.273972602739726, ('ขอบคุน', 'ขอบคุณ'): 0.25, ('ใหน', 'ไหน'): 0.23255813953488372, ('บช', 'บัญชี'): 0.23214285714285715, ('ใม่', 'ไม่'): 0.21875, ('อ่ะ', 'อะ'): 0.2124600638977636, ('ยังไง', 'อย่างไร'): 0.2116788321167883, ('ค่ะ', 'คะ'): 0.21063394683026584, ('อยุ่', 'อยู่'): 0.20754716981132076, 

In [ ]:
for key in sent_id_type:
  cou = 0
  print(f"=========[{key}]==========")
  for sent_id, words in sent_id_type[key]:
    cou += 1
    if cou == 20: break
    text = ds_tag.iloc[sent_id]['text']['input_ids'].squeeze(0).tolist()
    labels = ds_mlm.iloc[sent_id]['labels']
    text = [k for k in text if k != 1]
    labels = [k for k in labels if k != 1]
    original = [ids_to_tokens(text)]
    references = [[ids_to_tokens(labels)]]
    for i in words:
      print(f"MSP: {tokenizer.convert_ids_to_tokens(i[0])} => {tokenizer.convert_ids_to_tokens(i[1])}")
    print(original)
    print(references)
    print("-------------------")

=========[morphed]==========
MSP: มั๊ย => ไหม
[['<s>', '▁', 'จําเป็น', 'ต้อง', 'เปิดบริการ', 'กับ', 'สาขา', 'ที่', 'เ', 'ปิดบัญชี', '▁', 'มั๊ย', 'ครับ', '▁', 'หรือ', 'สาขา', 'ไหนก็ได้', '</s>']]
[[['<s>', '▁', 'จําเป็น', 'ต้อง', 'เปิดบริการ', 'กับ', 'สาขา', 'ที่', 'เ', 'ปิดบัญชี', '▁', 'ไหม', 'ครับ', '▁', 'หรือ', 'สาขา', 'ไหนก็ได้', '</s>']]]
-------------------
MSP: คับ => ครับ
[['<s>', '▁พี่', 'คับ', 'ผมอยาก', 'รุ้', 'ว่า', '</s>']]
[[['<s>', '▁พี่', 'ครับ', 'ผมอยาก', '▁รู้', 'ว่า', '</s>']]]
-------------------
MSP: ยังไง => อย่างไร
[['<s>', '▁แล้วถ้า', 'อยาก', 'เชค', 'ยอด', 'การใช้งาน', 'บัตรเครดิต', 'ต้องทํา', 'ยังไง', 'อะคะ', '</s>']]
[[['<s>', '▁แล้วถ้า', 'อยาก', 'เช็ค', 'ยอด', 'การใช้งาน', 'บัตรเครดิต', 'ต้องทํา', 'อย่างไร', 'อะคะ', '</s>']]]
-------------------
MSP: ตัง => สตางค์
[['<s>', '▁', 'สอบถาม', 'หน่อยค่ะ', '▁', 'บางทีก็', 'โอน', 'ตัง', 'ได้', '▁', 'บางทีก็', 'โอน', 'ไม่ได้', '▁', 'เปน', 'เพราะอะไร', 'ค่ะ', '</s>']]
[[['<s>', '▁', 'สอบถาม', 'หน่อยค่ะ', '▁', 'บางทีก็', 